# Task 2 - Data Exploration, Analysis, and Preprocessing 

This notebook covers data quality, integration, comprehensive exploration, and preparing the data for the modeling tasks.

## 2.1 Setup and Data Loading

In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the two primary datasets (Power Generation and Sensor Data for each plant)
plant1_gen = pd.read_csv('../data/raw/Plant_1_Generation_Data.csv')
plant1_weather = pd.read_csv('../data/raw/Plant_1_Weather_Sensor_Data.csv')
plant2_gen = pd.read_csv('../data/raw/Plant_2_Generation_Data.csv')
plant2_weather = pd.read_csv('../data/raw/Plant_2_Weather_Sensor_Data.csv')

## 2.2 Data Quality and Integration

### 2.2.1 Data Quality Assessment

#### A. General checks

##### Data Duplicates

In [ ]:
# Check for row duplicates in each dataset
print("Plant 1 Generation duplicates:", plant1_gen.duplicated().sum(), f"({round(plant1_gen.duplicated().sum()/len(plant1_gen)*100, 2)}%)")
print("Plant 2 Generation duplicates:", plant2_gen.duplicated().sum(), f"({round(plant2_gen.duplicated().sum()/len(plant2_gen)*100, 2)}%)")
print("Plant 1 Weather duplicates:", plant1_weather.duplicated().sum(), f"({round(plant1_weather.duplicated().sum()/len(plant1_weather)*100, 2)}%)")
print("Plant 2 Weather duplicates:", plant2_weather.duplicated().sum(), f"({round(plant2_weather.duplicated().sum()/len(plant2_weather)*100, 2)}%)")

In [ ]:
# Constants for 34 Days @ 15-min intervals
DAYS = 34
READINGS_PER_DAY = 24 * 4 # 96
EXPECTED_PER_SENSOR = DAYS * READINGS_PER_DAY

def analyse_dataset(df, plant_label="Plant 1", days=DAYS, readings_per_day=READINGS_PER_DAY):
    """
    Analyzes the dataset to compute expected vs actual readings per sensor/inverter.
    """
    inverter_count = df['SOURCE_KEY'].nunique()
    expected_readings = days * readings_per_day * inverter_count
    actual_readings = len(df)
    diff = actual_readings - expected_readings

    print(f"\n--- {plant_label} Generation Data ---")
    print(f"1. Unique Inverters identified: {inverter_count}")
    print(f"2. Expected Readings (Target):  {expected_readings:,}")
    print(f"   (Calculation: {days} days * {readings_per_day} readings * {inverter_count} inverters)")
    print(f"3. Actual Readings (Raw):       {actual_readings:,}")
    print(f"4. Conclusion:                  {diff:+,} Rows")


p1_gen_stats = analyse_dataset(plant1_gen, "Plant 1")
p2_gen_stats = analyse_dataset(plant2_gen, "Plant 2")
p1_weather_stats = analyse_dataset(plant1_weather, "Plant 1 Weather")
p2_weather_stats = analyse_dataset(plant2_weather, "Plant 2 Weather")

In [ ]:
# # Slice and do a visual inspection of the data to understand duplications

# # Define "Primary Keys" to make a row unique.
# key_columns = ['DATE_TIME', 'DC_POWER', 'SOURCE_KEY']

# # Find rows with duplicate Keys AND NOT duplicate Data
# has_duplicate_keys = plant1_gen.duplicated(subset=key_columns, keep=False)
# is_exact_duplicate = plant1_gen.duplicated(keep=False)

# # We only want rows where keys match, but data differs
# conflict_mask = has_duplicate_keys & ~is_exact_duplicate

# # Grab 1 random row to act as search parameter
# target_row = plant1_gen[conflict_mask].sample(1)

# # Retrieve the target row and its conflicts
# result = plant1_gen.merge(target_row[key_columns], on=key_columns)

# # Display result
# print(result)

**Conclusion:**
- When slicing data, we observe exact time stamps with all variables equal except for "operating_condition". Understanding this in detail and cleaning accordingly will be critical for the "Classification of Operating Conditions" task.

In [ ]:
## Check data types
def get_types_report(df):
    """Generate a report of pandas dtypes and unique Python types for each column in the DataFrame."""
    types = {}
    for col in df.columns:
        # Get unique python types in the column (useful for spotting mixed types)
        py_types = df[col].map(lambda x: type(x).__name__).unique().tolist()
        types[col] = py_types

    result_df = pd.DataFrame({
        'column': list(types.keys()),
        'pandas_dtype': [df[col].dtype for col in types.keys()],
        'python_types': [types[col] for col in types.keys()]
    })
    return result_df

# Get unique Python types and pandas dtypes for the four datasets
print("Plant 1 Generation - Data Types Report:")
print(get_types_report(plant1_gen), "\n")

print("Plant 1 Weather - Data Types Report:")
print(get_types_report(plant1_weather), "\n")

print("Plant 2 Generation - Data Types Report:")
print(get_types_report(plant2_gen), "\n")

print("Plant 2 Weather - Data Types Report:")
print(get_types_report(plant2_weather))

**Conclusion:**
- Mixed formats: In "Plant 1 - Generation data" the variable "Operating_Condition" mariable exhibits mixed data types, strings and float. This means there are probably missing values in this column (NaN)

- Integers for categorical information: Variable Plant_ID is currently encoded as an integer across, is shown as an integer. However, it serves as a unique categorical identifier rather than a quantitative metric. This distinction must be explicitly noted during modeling to ensure algorithms do not misinterpret the ID as having numerical magnitude or order

#### B. Missing Values

In [ ]:
## Check for missing values in the four data sets

def get_missing_data_report(df):
    """Generate a report of missing data percentages for each column in the DataFrame."""
    missing_data_report = pd.DataFrame({
        'Columns': df.columns,
        'Missing Values': df.isna().sum().values,
        'Percentage Missing': ((df.isna().sum().values / len(df)) * 100).round(2)
    })
    return missing_data_report

# Generate and print missing data report for the datasets
print("Power Generation 1 - Missing Values Report:")
print(get_missing_data_report(plant1_gen),"\n")
print("Weather Sensor 1 - Missing Values Report:")
print(get_missing_data_report(plant1_weather))
print("Power Generation 2 - Missing Values Report:")
print(get_missing_data_report(plant2_gen),"\n")
print("Weather Sensor 2- Missing Values Report:")
print(get_missing_data_report(plant2_weather))

In [ ]:
## Check for missing data ranges (rows) in the four data sets
# -------------------------------------------------------
# Generate a report
# -------------------------------------------------------
def get_time_gap_report(df):
    """
    Generate a report of missing time intervals in the DataFrame.
    Assumes 'DATE_TIME' column exists.
    """
    # Ensure datetime and sort
    df = df.copy()
    df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])
    df = df.sort_values('DATE_TIME')
    
    # Create the expected grid for the intervals (perfect 15 min intervals)
    start = df['DATE_TIME'].min()
    end = df['DATE_TIME'].max()
    expected_range = pd.date_range(start=start, end=end, freq='15min')
    
    # Calculate statistics
    expected_count = len(expected_range)
    actual_count = df['DATE_TIME'].nunique()
    missing_count = expected_count - actual_count
    pct_missing = (missing_count / expected_count) * 100
    
    # Create Report DataFrame
    report = pd.DataFrame({
        'Metric': ['Start Time', 'End Time', 'Expected Intervals', 'Actual Intervals', 'Missing Intervals', '% Missing'],
        'Value': [start, end, expected_count, actual_count, missing_count, round(pct_missing, 2)]
    })
    return report

# Generate and print time gap reports
print("Power Generation 1 - Time Gap Report:")
print(get_time_gap_report(plant1_gen),"\n")

print("Weather Sensor 1 - Time Gap Report:")
print(get_time_gap_report(plant1_weather),"\n")

print("Power Generation 2 - Time Gap Report:")
print(get_time_gap_report(plant2_gen),"\n")

print("Weather Sensor 2 - Time Gap Report:")
print(get_time_gap_report(plant2_weather))

**Conclusion:** 

Missing values (NaN):
- Plant 1: Generation data exhibits missing values. There are about 2.3% missing values in the Operating_Condition column of the data set. On the contrary, weather data shows 0 missing values (empty cells)
- Plant 2: There are no missing values in the generation or weather data.

Missing temporal datetimes (entire rows):
- Plant 1: Is the only one with a time range different from the other datasets. We suspect the date format is not consistent and it is also the reason why the missing value count is high (around 90%) thus needs to be fixed.
- Plant 2: The datetime coverage is nearly perfect, it only misses 5 intervals (0.15%) of information.

#### C. Inconsistencies & Anomalies

##### Date Format Fixes

In [ ]:
# Ensure all time columns are datetime objects
plant1_weather['DATE_TIME'] = pd.to_datetime(plant1_weather['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')
plant2_weather['DATE_TIME'] = pd.to_datetime(plant2_weather['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')
plant2_gen['DATE_TIME'] = pd.to_datetime(plant2_gen['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
# The timerange of the data is known to be within May 15 and June 17
print("Data timerange:", plant1_weather['DATE_TIME'].min(), "to", plant1_weather['DATE_TIME'].max())

In [ ]:
# Resolve DATE_TIME format ambiguity in Plant 1 Generation Data
# Extract month and day parts from DATE_TIME
plant1_gen = plant1_gen.join(plant1_gen['DATE_TIME'].str.extract(r'-(\d{2})-(\d{2}) ')).rename(columns={0: 'mid', 1: 'right'})

# Determine date format based on extracted parts knowing that the timerange of the data is known to be within May 15 and June 17
plant1_gen['date_format'] = ''
plant1_gen.loc[(plant1_gen['mid']=='05') & (plant1_gen['right']=='06'), "date_format"] = 'dd-mm'
plant1_gen.loc[(plant1_gen['mid']=='06') & (plant1_gen['right']=='05'), "date_format"] = 'mm-dd'
plant1_gen.loc[((plant1_gen['mid']=='05') | (plant1_gen['mid']=='06')) & (plant1_gen['date_format'] == ''), "date_format"] = 'mm-dd'
plant1_gen.loc[((plant1_gen['right']=='05') | (plant1_gen['right']=='06')) & (plant1_gen['date_format'] == ''), "date_format"] = 'dd-mm'

# Convert DATE_TIME to datetime using the determined format
mask_ddmm = plant1_gen['date_format'] == 'dd-mm'
mask_mmdd = plant1_gen['date_format'] == 'mm-dd'
plant1_gen.loc[mask_ddmm,"DATE_TIME"] = pd.to_datetime(plant1_gen.loc[mask_ddmm,"DATE_TIME"], format='%Y-%d-%m %H:%M:%S')
plant1_gen.loc[mask_mmdd,"DATE_TIME"] = pd.to_datetime(plant1_gen.loc[mask_mmdd,"DATE_TIME"], format='%Y-%m-%d %H:%M:%S')
plant1_gen['DATE_TIME'] = pd.to_datetime(plant1_gen['DATE_TIME']) # Final conversion to datetime
plant1_gen = plant1_gen.drop(columns=['mid', 'right', 'date_format'])

In [ ]:
# Generate and print the time gap report again for Plant 1 Generation Data (after fixing DATE_TIME format)
print("Power Generation 1 - Time Gap Report:")
print(get_time_gap_report(plant1_gen),"\n")

With the right date format correction, plant generation 1 presents a 3.25% of missing date intervals (instead of 90%+ before the fix).

##### Power Format Fixes

In [ ]:
print("Power Generation Plant 1\n", plant1_gen[['DC_POWER','AC_POWER']].describe(),"\n")
print("Power Generation Plant 2\n", plant2_gen[['DC_POWER','AC_POWER']].describe())

From the observation of the DC and AC power generation descriptive statistics, we notice that Plant 1's DC power values are an order of magnitude higher than Plant 2's. To fix this error in the data, we scale down Plant 1's DC power values by a factor of 10 to ensure consistency across both plants.

In [ ]:
# Fix DC_POWER scale issue in Plant 1 Generation Data
plant1_gen['DC_POWER'] = plant1_gen['DC_POWER'] / 10

In [ ]:
# Reprint descriptive statistics after fixing DC_POWER scale issue
print("Power Generation Plant 1\n", plant1_gen[['DC_POWER','AC_POWER']].describe(),"\n")
print("Power Generation Plant 2\n", plant2_gen[['DC_POWER','AC_POWER']].describe())

Now the power values look consistent between both plants.

##### Further Checks

In [ ]:
## Inconsistencies are checked for different situations for all data sets
# Grouping for iteration
gen_data = [("Plant 1", plant1_gen), ("Plant 2", plant2_gen)]
weather_data = [("Plant 1", plant1_weather), ("Plant 2", plant2_weather)]


## Part A: Generation data
print("\n--- POWER GENERATION CHECKS ---")

# CHECK 1: Efficiency violation 
# Impossible for AC Output > DC Input (inverter efficiency can't be > 100%)

print("\n[Gen Check 1] Efficiency violation (AC > DC)")
for name, df in gen_data:
    # We allow a tiny buffer (0.1 kW) for sensor timing mismatch
    errors = df[df['AC_POWER'] > df['DC_POWER'] + 0.1]
    print(f"  {name}: {len(errors)} rows failed.")

# CHECK 2: Negative power
# Solar panels cannot consume power (cannot be negative)
print("\n[Gen Check 2] Negative power (AC or DC < 0)")
for name, df in gen_data:
    errors = df[(df['AC_POWER'] < 0) | (df['DC_POWER'] < 0)]
    print(f"  {name}: {len(errors)} rows failed.")


## Part B: Weather data

print("\n--- WEATHER CHECKS ---")

# CHECK 1: No irradiance at night
# Irradiance > 0 when it is dark (10 PM - 4 AM short range assuming its summer)
print("\n[Weather Check 1] Night irradiance (Irradiation > 0 at Night)")
for name, df in weather_data:
    hour = df['DATE_TIME'].dt.hour
    night_mask = (hour >= 22) | (hour < 4)
    errors = df[night_mask & (df['IRRADIATION'] > 0)]
    print(f"  {name}: {len(errors)} rows failed.")

# CHECK 2: Panel being hot without irradiance at night
# No Sun (0 Irr) but Module is significantly hotter than Ambient
print("\n[Weather Check 2] Hot panel at night (Mod Temp > Amb Temp + 5°C w/ no Sun)")
for name, df in weather_data:
    # If Irradiance is 0, Module shouldn't be hot compared to air
    errors = df[(df['IRRADIATION'] == 0) & 
                (df['MODULE_TEMPERATURE'] > df['AMBIENT_TEMPERATURE'] + 5.0)]
    print(f"  {name}: {len(errors)} rows failed.")

# CHECK 3: Broken sensors
# Ambient Temp stays exactly the same for 4 consecutive readings (1 hour). Testing for a extreme case.
print("\n[Weather Check 3] Broken sensors (Ambient Temp frozen for 1hr)")
for name, df in weather_data:
    df_sorted = df.sort_values(by='DATE_TIME') 
    
    # True if current value equals previous value
    is_frozen = df_sorted['AMBIENT_TEMPERATURE'].diff() == 0
    
    # Rolling sum: if 4 consecutive rows are "True", we have a 1-hour freeze
    frozen_blocks = is_frozen.rolling(4).sum() == 4
    print(f"  {name}: {frozen_blocks.sum()} frozen sequences found.")

**Conclusion:** 

Generation data
- Plant 1: There are 396 instances (~0.6%) of efficiency violations (system reports that AC output is higher than DC, which is physically imposible).
- Plant 2: There are 396 instances (~0.6%) of efficiency violations (system reports that AC output is higher than DC, which is physically imposible).

Weather data
- Plant 1: Passed all consistency checks
- Plant 2: 24 instances (~0.7%) of "Night Sun" (Irradiance > 0 at night) This culd be due to the sunrise/sunset exact timings or callibration

### 2.2.2 Data Handling

In [ ]:
# SHOULD WE DROP DUPLICATES??? ask Panagiotis Angeloudis
def recalculate_yields(df, drop_dup=True):
    df = df.copy()
    if drop_dup:
        df.drop_duplicates(subset=['DATE_TIME', 'SOURCE_KEY'], inplace=True)
    df.sort_values(by=['DATE_TIME', 'SOURCE_KEY'], inplace=True)

    # day/month for per-day grouping
    df['day'] = df['DATE_TIME'].dt.day
    df['month'] = df['DATE_TIME'].dt.month

    # DAILY_YIELD: cumulative AC_POWER per SOURCE_KEY per day
    df['DAILY_YIELD'] = df.groupby(['SOURCE_KEY', 'month', 'day'])['AC_POWER'].cumsum()

    # ensure TOTAL_YIELD column exists for the earliest-yield lookup
    if 'TOTAL_YIELD' not in df.columns:
        df['TOTAL_YIELD'] = np.nan

    # get earliest DATE_TIME per SOURCE_KEY and corresponding TOTAL_YIELD
    min_dates = df.groupby('SOURCE_KEY', as_index=False)['DATE_TIME'].min()
    first_yield = (min_dates
                   .merge(df[['SOURCE_KEY', 'DATE_TIME', 'TOTAL_YIELD']],
                          on=['SOURCE_KEY', 'DATE_TIME'],
                          how='left')
                   [['SOURCE_KEY', 'DATE_TIME', 'TOTAL_YIELD']]
                   .drop_duplicates('SOURCE_KEY')
                   .rename(columns={'DATE_TIME': 'FIRST_DATE_TIME', 'TOTAL_YIELD': 'FIRST_TOTAL_YIELD'}))

    # merge FIRST_TOTAL_YIELD and fill missing with 0
    df = df.merge(first_yield[['SOURCE_KEY', 'FIRST_TOTAL_YIELD']], on='SOURCE_KEY', how='left')
    df['FIRST_TOTAL_YIELD'] = df['FIRST_TOTAL_YIELD'].fillna(0.0)

    # recompute TOTAL_YIELD
    df['TOTAL_YIELD'] = df['DAILY_YIELD'] + df['FIRST_TOTAL_YIELD']

    # cleanup helper columns
    df.drop(columns=['day', 'month'], inplace=True)
    return df

# Apply to both plants and assign back
plant1_gen_ = recalculate_yields(plant1_gen)
plant2_gen_ = recalculate_yields(plant2_gen)

### 2.2.3 Integration

## 2.3 Exploratory Data Analysis

### 2.3.1 Statistical Summary

### 2.3.2 Visualizations

### 2.3.3 Trend Analysis

### 2.3.4 Correlation Analysis

### 2.3.5 Pattern Identification

## 2.4 Feature Engineering

### 2.4.1 Feature Scaling

### 2.4.2 Feature Selection